In [2]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{||\mathbf{x} - \mathbf{y}||}
$$

In [24]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    y = np.zeros(len(vec))
    y[0] = np.linalg.norm(vec)
    u = vec + np.sign(vec[0])*y
    u /= np.linalg.norm(u)
    
    H = np.diag(np.ones(len(vec))) - 2 * np.outer(u, u)
    outvec = H @ vec
    
    return outvec, H

In [25]:
# Test (marked)

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1)

In [26]:
rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the QR decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $QR$ decomposition of $\mathbf{A}$. 

In [69]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    
    q = np.diag(np.ones((m)))
    r = a1
    for i in range(n):
        
        H = np.diag(np.ones((m)))
        H[i:, i:] = householder(a1[i:, i])[1]
        q = q @ H
        r = H @ r
    return q, r

In [70]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [77]:
# Test (marked)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [84]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

print('q: \n', q, '\n')
print('qq: \n', qq, '\n\n')
print('r: \n', r, '\n')
print('rr: \n', rr, '\n')

q: 
 [[-0.13665049  0.84904838 -0.09632583 -0.49189125 -0.09595741]
 [-0.56035895 -0.14917522  0.7241077  -0.17977948 -0.32724911]
 [-0.19725922 -0.43633977 -0.56781974 -0.50038654 -0.4448488 ]
 [-0.62498418  0.20638973 -0.3631482   0.63542196 -0.17652298]
 [-0.48765568 -0.15451371 -0.10997042 -0.26753425  0.80910406]] 

qq: 
 [[-0.13665049  0.53601299  0.09369752  0.661619   -0.49749149]
 [-0.56035895  0.0935397   0.53326881 -0.52477245 -0.34276292]
 [-0.19725922  0.65948912 -0.60068463 -0.37879015  0.14784752]
 [-0.62498418 -0.50418303 -0.52144688  0.18967657 -0.21750907]
 [-0.48765568  0.12171264  0.27224305  0.32774225  0.75222783]] 


r: 
 [[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.02568624 -0.04089491]
 [-0.         -0.15877435 -0.61148064]
 [-0.         -0.8107868  -0.52447249]
 [ 0.         -0.15816707 -0.34630181]] 

rr: 
 [[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.          0.         

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

В первом задании у нас была свобода выбрать плюс или минус в подсчете  $\mathbf{u}$, мы рпедпочли знак первоой компоненты вектора  $\mathbf{x}$, чтобы не было большой ошибки при работе с близкими к  $\mathbf{x}$ векторами, а что там выбирает машина, мы не знаем, в любом случе, все компенсируется при умножении. Вот так объясняется несоответствие в знаках некоторых элементов матриц.

Еще есть некоторые элементы, которые вообще не похожи, что же, Википедия утверждает, что в общем случае QR-разложение не единственно, вот они и разные, получается:) 

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. Write a second function, which uses the reflection vertices to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests.

In [102]:
def QR(A):
    """Constructs the QR decomposition of matrix.
    
    Parameters
    ----------
    A : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    R : ndarray, shape(m, m)
        The orthogonal matrix
    U : ndarray, shape(m, n)
        The upper triangular matrix
    """
    
    R = np.array(A)
    r = np.array(A)
    m, n = A.shape
    U = np.zeros((m,n))
    
    for i in range(n):
        
        vec = r[:,0]    
        y = np.zeros(len(vec))
        y[0] = np.linalg.norm(vec)
        u = vec + np.sign(vec[0])*y
        u /= np.linalg.norm(u)
        u_ = np.zeros(m)
        u_[i:] = u
        U[:,i] = u_
        R -= 2*(u_[:,None] @ u_[:,None].T @ R)
        r = R[i+1:,i+1:]
    
    return R, U

def mmult(A, u):
    """Multiplies a matrix with 𝐐𝑇
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    u : ndarray, shape(m, n)
        Reflection vectors
    
    Returns
    -------
    R : ndarray, shape(m, n)
        Multiplied matrix
    """
    
    R = np.array(A)
    m, n = a.shape
    for i in range (n):
        R -= 2 * ((u[:,i])[:,None] @ (u[:,i])[:,None].T @ R)
    return R


In [115]:
# Test

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))

r, q = QR(a)
print('\n QR: \n', r)

print('\n QT: \n', mmult(a,q))
print('\n scipy qr:\n', rr)


 QR: 
 [[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.         -0.          0.        ]
 [-0.         -0.         -0.        ]]

 QT: 
 [[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.         -0.          0.        ]
 [-0.         -0.         -0.        ]]

 scipy qr:
 [[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]


Как удивительно и приятно все совпало